In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML # dùng để hiển thị 
%matplotlib inline 
import helper

#All the header files required for the code
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random

In [2]:
nan = np.nan # not a number đại diện cho các sản phẩm chưa được đánh giá
ratings_matrix = np.array([[2, 3, 5, nan, 1, 4], 
                           [nan, 3, nan, 4, nan, 3],
                           [3, 5, nan, 2, 4, nan],
                           [3, 3, 4, nan, 3, nan]])

ratings_matrix = pd.DataFrame(ratings_matrix)
ratings_matrix.head()

,0,1,2,3,4,5
0,2.0,3.0,5.0,NaN,1.0,4.0
1,NaN,3.0,NaN,4.0,NaN,3.0
2,3.0,5.0,NaN,2.0,4.0,NaN
3,3.0,3.0,4.0,NaN,3.0,NaN


In [3]:
ratings_matrix.index += 1
ratings_matrix.index.name = 'UserID'

ratings_matrix.columns += 1
ratings_matrix.columns.name = 'MovieID'
ratings_matrix.head()

MovieID,1,2,3,4,5,6
UserID,,,,,,
1,2.0,3.0,5.0,NaN,1.0,4.0
2,NaN,3.0,NaN,4.0,NaN,3.0
3,3.0,5.0,NaN,2.0,4.0,NaN
4,3.0,3.0,4.0,NaN,3.0,NaN


In [4]:
ratings_matrix.fillna(0,inplace=True)
ratings_matrix.head()

MovieID,1,2,3,4,5,6
UserID,,,,,,
1,2.0,3.0,5.0,0.0,1.0,4.0
2,0.0,3.0,0.0,4.0,0.0,3.0
3,3.0,5.0,0.0,2.0,4.0,0.0
4,3.0,3.0,4.0,0.0,3.0,0.0


In [5]:
# matrix bias
ratings_matrix.mean(axis=0)

MovieID
1    2.00
2    3.50
3    2.25
4    1.50
5    2.00
6    1.75
dtype: float64

In [6]:
# SD Z−scores
ratings_matrix.std()

MovieID
1    1.414214
2    1.000000
3    2.629956
4    1.914854
5    1.825742
6    2.061553
dtype: float64

In [7]:
# matrix đã khử bias
ratings_demeaned = ratings_matrix.sub(ratings_matrix.mean(axis=0), axis=1)/ratings_matrix.std()
ratings_demeaned.head()

MovieID,1,2,3,4,5,6
UserID,,,,,,
1,0.000000,-0.5,1.045645,-0.783349,-0.547723,1.091410
2,-1.414214,-0.5,-0.855528,1.305582,-1.095445,0.606339
3,0.707107,1.5,-0.855528,0.261116,1.095445,-0.848875
4,0.707107,-0.5,0.665410,-0.783349,0.547723,-0.848875


In [8]:
# # SVD
# import numpy as np                                                                             
# from scipy import linalg
# from scipy.sparse import linalg as slinalg

# from numpy import linalg as LA

# u, s, v = LA.svd(ratings_demeaned)
# print(u) 
# print(s) 
# print(v) 

In [9]:
# s = np.diag(s)
# all_user_predicted_ratings = np.dot(np.dot(u, s), v)

In [14]:
# TruncatedSVD
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=2)
X_reduced = svd.fit_transform(ratings_demeaned)
print(X_reduced)

[[ 0.76826105  1.57430268]
 [ 2.17083812 -1.21264465]
 [-1.83577416 -1.39798571]
 [-1.10332501  1.03632768]]


In [10]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(ratings_demeaned, k=2)
print(U) 
print(sigma) 
print(Vt) 

[[-0.5959987  -0.24429088]
 [ 0.45908239 -0.69028092]
 [ 0.52924871  0.58373762]
 [-0.3923324   0.35083418]]
[2.64145321 3.14486184]
[[-0.20913718  0.40072541 -0.65487052  0.57232666  0.07133043 -0.18487715]
 [ 0.52054646  0.3712326   0.02199077 -0.26463985  0.54742653 -0.47013201]]


In [11]:
sigma = np.diag(sigma)
print(sigma)

[[2.64145321 0.        ]
 [0.         3.14486184]]


In [12]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
# all_user_predicted_ratings = np.dot(U, sigma)
print(all_user_predicted_ratings)

[[-0.07067034 -0.91606664  1.01406975 -0.6977029  -0.53286216  0.6522367 ]
 [-1.38363117 -0.31994835 -0.84186363  1.26851912 -1.10187591  0.79639019]
 [ 0.66323494  1.24170761 -0.87512953  0.3142855   1.10467039 -1.12151179]
 [ 0.79106658 -0.00569262  0.70292341 -0.88510172  0.53006768 -0.3271151 ]]


In [13]:
preds = pd.DataFrame(all_user_predicted_ratings)
preds.head()

,0,1,2,3,4,5
0,-0.070670,-0.916067,1.014070,-0.697703,-0.532862,0.652237
1,-1.383631,-0.319948,-0.841864,1.268519,-1.101876,0.796390
2,0.663235,1.241708,-0.875130,0.314285,1.104670,-1.121512
3,0.791067,-0.005693,0.702923,-0.885102,0.530068,-0.327115
